# Running open-source LLMs on CPU local environment using Langchain

- **Downloading small language models (SLM) from Huggingface**

Huggingface offers large range of open-source language models in .gguf formats and llamafile formats. The LLM in these formats are executable using llamafile command.

Starting the model without --nobrowser option allows to show the running model over a browser.

In [ ]:
# Download a llamafile from HuggingFace
wget https://huggingface.co/jartine/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# Make the file executable. On Windows, instead just rename the file to end in ".exe".
chmod +x TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile

# Start the model server. Listens at http://localhost:8080 by default.
./TinyLlama-1.1B-Chat-v1.0.Q5_K_M.llamafile --server (--nobrowser)

- **Formatting the prompt from my prompt data file**

Prompt generation using the langchain template

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        "Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

prompt

PromptTemplate(input_variables=['language', 'topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [13]:
first_prompt = prompt.format(topic="sports", language="english")

- **Evoking LLM**

In [14]:
from langchain_community.llms.llamafile import Llamafile

llm = Llamafile()

response = llm.invoke(first_prompt)


In [18]:
import pprint
pprint.pprint(response)

(':\n'
 '(1) What do you call a football player who never scores? A fumble.\n'
 '(2) How did the golfer manage to break 140? By missing the green and holing '
 'the putt.\n'
 '(3) Who won more gold medals than any other athlete? Gymnast Simone Biles.\n'
 '(4) Can you make a football punny joke? No, but I can tell you that when '
 'football players go on vacation, they bring their cleats with them. (5) What '
 'do you call a tennis player who never wins a tournament? A loser.\n'
 '(6) Who scored the most points in the NBA finals? The Chicago Bulls.\n'
 "(7) What is the hardest thing to hit while playing soccer? The goalie's "
 'hands.\n'
 '(8) Can you come up with a football punny joke about the NFL season? No, but '
 'I can tell you that the Washington Redskins lost all four of their games '
 'this weekend. (9) How did the golf player finish second in the Masters? By '
 'missing the green and making double bogey.\n'
 '(10) Can you make a tennis punny joke for me? No, but I will tell yo

- **Summarizing a document using langchain**

In [20]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Under an updated policy, the LCSD aims to provide more performance slots for arts groups and establish clearer venue identities, 'thereby fostering a diversified, professional, industry and mega-event development of performing arts,'a Thursday statement read. The Hong Kong Coliseum in Hung Hom, previously given priority for sports events, will be repositioned to prioritise concerts, thereby promoting the development of a “concert economy,” the department said.", 
             metadata={"title": "HK_news1"}),

    Document(page_content="South Korea finds itself in the middle of a growing superpower standoff between the United States, its traditional security guarantor, and China, its largest trade partner but also a key ally of North Korea. The two men – identified as international students in Busan – were arrested on Wednesday, charged with injuring the military interests of South Korea and violating the Protection of Military Bases and Installations Act, according to the Busan Metropolitan Police.", 
             metadata={"title": "HK_news2"}),

    Document(page_content="Hong Kong lawmakers have passed amendments to the city’s union laws, banning anyone convicted of national security offences from serving in trade unions for life. The amendment bill, passed by a show of hands in the opposition-free legislature on Wednesday, contains a slew of new terms to tighten union laws on national security grounds, including a requirement for foreign funding to be vetted by authorities. The changes to the Trade Unions Ordinance were proposed in February “to better fulfil the duty of safeguarding national security” under the city’s two national security laws.", 
             metadata={"title": "HK_news3"}),
]

In [32]:
documents[0].page_content

"Under an updated policy, the LCSD aims to provide more performance slots for arts groups and establish clearer venue identities, 'thereby fostering a diversified, professional, industry and mega-event development of performing arts,'a Thursday statement read. The Hong Kong Coliseum in Hung Hom, previously given priority for sports events, will be repositioned to prioritise concerts, thereby promoting the development of a “concert economy,” the department said."

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
chain = create_stuff_documents_chain(llm, prompt, 
                                     )

result = chain.invoke({"context": documents},
                      config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:stuff_documents_chain] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context>] Entering Chain run with input:
[inputs]
[chain/start] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] Entering Chain run with input:
[inputs]
[chain/end] [chain:stuff_documents_chain > chain:format_inputs > chain:RunnableParallel<context> > chain:format_docs] s] Exiting Chain run with output:
{
  "output": "Under an updated policy, the LCSD aims to provide more performance slots for arts groups and establish clearer venue identities, 'thereby fostering a diversified, professional, industry and mega-event development of performing arts,'a Thursday statement read. The Hong Kong Coliseum in Hung Hom, previously given priority for sports events, wi

In [34]:
pprint.pprint(result)

('\n'
 '\n'
 'A new report from the UN’s International Labour Organization (ILO) shows '
 'that while 42 per cent of all jobs created worldwide since the 1980s have '
 'been in the informal sector, only 37 per cent of those workers were covered '
 'by formal labour laws. The ILO’s report, titled ‘Limited Employment '
 'Opportunities for Informal Workers’, found that “the situation of informal '
 'workers is still poor,” particularly in developing countries and in “the '
 'majority” of African countries where the vast majority of people live.\n'
 '\n'
 'The United States will impose new sanctions on North Korea’s top nuclear '
 'scientist on Thursday, the White House said, after Pyongyang conducted a '
 'series of missile tests and its nuclear test last month triggered a UN '
 'Security Council resolution demanding the dismantling of a nuclear program. '
 'The sanctions are aimed at blocking money and technology that could be used '
 'to develop nuclear weapons or other weapons of mass 